In [1]:
import pandas as pd
import os
import shutil
import re
from pathlib import Path
from datetime import datetime

class EventProcessorComplete:
    """Полный класс для обработки мероприятий: разделение, конвертация и форматирование"""
    
    def __init__(self, input_file, csv_dir="split_events", excel_dir="excel_events", 
                 final_dir="4.excel_final", default_category="Hard skills"):
        """
        Инициализация процессора
        
        Args:
            input_file: Исходный файл (CSV или Excel)
            csv_dir: Папка для промежуточных CSV файлов
            excel_dir: Папка для неформатированных Excel файлов  
            final_dir: Папка для финальных отформатированных Excel файлов
            default_category: Категория по умолчанию для заполнения
        """
        self.input_file = input_file
        self.csv_dir = csv_dir
        self.excel_dir = excel_dir
        self.final_dir = final_dir
        self.default_category = default_category
        self.csv_files = []
        
        # Настройки для чтения файлов
        self.encodings = ['utf-8-sig', 'utf-8', 'cp1251', 'windows-1251']
        self.separators = [';', ',', '\t']
    
    def _ensure_folder(self, folder_path):
        """Создает папку если она не существует"""
        Path(folder_path).mkdir(parents=True, exist_ok=True)
    
    def _normalize_filename(self, filename):
        """Очищает название файла от недопустимых символов"""
        if not isinstance(filename, str):
            return "unnamed_file"
        filename = re.sub(r'[<>:"/\\|?*\n\r\t\x00-\x1f]', '_', filename)
        filename = re.sub(r'_+', '_', filename).strip('._')
        return filename[:100] if filename else "unnamed_file"
    
    def _read_file_with_encoding(self, file_path, **kwargs):
        """Читает файл с автоопределением кодировки и разделителя"""
        ext = os.path.splitext(file_path)[1].lower()
        
        if ext in ['.xlsx', '.xls']:
            return pd.read_excel(file_path, **kwargs)
        
        elif ext == '.csv':
            # Пробуем разные кодировки и разделители
            for encoding in self.encodings:
                for sep in self.separators:
                    try:
                        df = pd.read_csv(file_path, encoding=encoding, sep=sep, **kwargs)
                        if len(df.columns) > 1 or len(df) == 0:
                            return df
                    except:
                        continue
            raise Exception(f"Не удалось прочитать файл: {file_path}")
        else:
            raise Exception(f"Неподдерживаемый формат: {ext}")
    
    def _extract_base_name(self, event_name):
        """Извлекает базовое название мероприятия без номеров групп и занятий"""
        if not isinstance(event_name, str):
            return ""
        
        # Убираем "- Занятие X"
        if " - Занятие " in event_name:
            base_name = event_name.split(" - Занятие ")[0]
        elif " - занятие " in event_name.lower():
            base_name = event_name.split(" - занятие ")[0]
        else:
            base_name = event_name
            
        # Убираем "Группа X" и номера в начале
        base_name = re.sub(r'\s+Группа\s+\d+', '', base_name, flags=re.IGNORECASE)
        base_name = re.sub(r'^\d+\.\s*', '', base_name)
        return base_name.strip()
    
    def _get_sort_key(self, event_name):
        """Создает ключ для сортировки: группа -> программа -> занятия"""
        if not isinstance(event_name, str):
            return (0, 0, 1)
            
        group_match = re.search(r'[Гг]руппа\s+(\d+)', event_name)
        session_match = re.search(r'Занятие\s+(\d+)', event_name, re.IGNORECASE)
        
        group_num = int(group_match.group(1)) if group_match else 0
        session_num = int(session_match.group(1)) if session_match else 0
        sort_type = 0 if "Занятие" not in event_name else 1
        
        return (group_num, sort_type, session_num)
    
    def _format_date(self, date_str):
        """Форматирует дату с добавлением двойного пробела"""
        if pd.isna(date_str):
            return date_str
        
        if isinstance(date_str, datetime):
            return date_str.strftime('%d.%m.%Y  %H:%M')
        
        if isinstance(date_str, str):
            date_str = date_str.strip()
            
            # Основные форматы
            formats = ['%d.%m.%Y %H:%M', '%d.%m.%Y  %H:%M', '%d.%m.%Y %H:%M:%S', 
                      '%Y-%m-%d %H:%M', '%d.%m.%Y', '%Y-%m-%d']
            
            for fmt in formats:
                try:
                    date_obj = datetime.strptime(date_str, fmt)
                    return date_obj.strftime('%d.%m.%Y  %H:%M')
                except ValueError:
                    continue
            
            # Регулярные выражения для нестандартных форматов
            match = re.match(r'(\d{1,2})\.(\d{1,2})\.(\d{4})\s+(\d{1,2}):(\d{1,2})', date_str)
            if match:
                day, month, year, hour, minute = match.groups()
                date_obj = datetime(int(year), int(month), int(day), int(hour), int(minute))
                return date_obj.strftime('%d.%m.%Y  %H:%M')
        
        return date_str
    
    def split_events(self):
        """Этап 1: Разделяет основной файл на отдельные CSV файлы по мероприятиям"""
        print("🔄 Этап 1: Разделение мероприятий...")
        self._ensure_folder(self.csv_dir)
        
        if not os.path.exists(self.input_file):
            raise Exception(f"Файл не найден: {self.input_file}")
        
        df = self._read_file_with_encoding(self.input_file)
        
        # Проверяем обязательные столбцы
        required_cols = ['Название мероприятия', 'Тип мероприятия']
        missing = [col for col in required_cols if col not in df.columns]
        if missing:
            raise Exception(f"Отсутствуют столбцы: {missing}")
        
        # Группируем строки по базовому названию мероприятия
        event_groups = {}
        for idx, row in df.iterrows():
            base_name = self._extract_base_name(row["Название мероприятия"])
            if base_name:
                if base_name not in event_groups:
                    event_groups[base_name] = []
                event_groups[base_name].append(idx)
        
        # Создаем отдельный файл для каждого мероприятия
        self.csv_files = []
        for base_name, indices in event_groups.items():
            event_df = df.iloc[indices].copy()
            
            # Сортируем: программы сначала, потом занятия по номерам
            event_df['sort_key'] = event_df.apply(
                lambda row: self._get_sort_key(row["Название мероприятия"]), axis=1
            )
            event_df = event_df.sort_values('sort_key').drop('sort_key', axis=1)
            
            # Создаем безопасное имя файла
            safe_name = self._normalize_filename(base_name)
            file_path = os.path.join(self.csv_dir, f"{safe_name}.csv")
            
            # Избегаем дублирования имен файлов
            counter = 1
            original_path = file_path
            while os.path.exists(file_path):
                file_path = f"{original_path.replace('.csv', '')}_{counter}.csv"
                counter += 1
            
            event_df.to_csv(file_path, index=False, encoding='utf-8-sig', sep=';')
            self.csv_files.append(file_path)
        
        print(f"✅ Создано {len(self.csv_files)} CSV файлов")
        return len(self.csv_files)
    
    def convert_to_excel(self):
        """Этап 2: Конвертирует CSV файлы в Excel формат"""
        print("🔄 Этап 2: Конвертация в Excel...")
        self._ensure_folder(self.excel_dir)
        
        # Исключаем служебные файлы (начинающиеся с _)
        csv_files = [f for f in self.csv_files if not os.path.basename(f).startswith('_')]
        converted = 0
        
        for csv_file in csv_files:
            try:
                df = self._read_file_with_encoding(csv_file)
                base_name = os.path.splitext(os.path.basename(csv_file))[0]
                excel_file = os.path.join(self.excel_dir, f"{base_name}.xlsx")
                
                with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
                    df.to_excel(writer, index=False, sheet_name='Данные')
                converted += 1
                
            except Exception as e:
                print(f"⚠️  Ошибка конвертации {csv_file}: {e}")
        
        print(f"✅ Конвертировано {converted} из {len(csv_files)} файлов")
        return converted
    
    def format_excel_files(self, format_dates=True, fill_categories=True):
        """Этап 3: Форматирует Excel файлы (даты и категории)"""
        print("🔄 Этап 3: Форматирование Excel файлов...")
        self._ensure_folder(self.final_dir)
        
        # Получаем список Excel файлов
        excel_files = [f for f in os.listdir(self.excel_dir) 
                      if f.lower().endswith(('.xlsx', '.xls'))]
        
        if not excel_files:
            print("⚠️  Excel файлы для форматирования не найдены")
            return 0
        
        print(f"📁 Найдено файлов: {len(excel_files)}")
        if format_dates:
            print("📅 Форматирование дат: включено")
        if fill_categories:
            print(f"🏷️  Заполнение категорий: {self.default_category}")
        
        # Обрабатываем файлы
        successful = 0
        for filename in excel_files:
            source_path = os.path.join(self.excel_dir, filename)
            target_path = os.path.join(self.final_dir, filename)
            
            try:
                df = pd.read_excel(source_path)
                modified = False
                
                # Обработка дат
                if format_dates:
                    date_columns = ['Дата начала обучения', 'Дата окончания обучения']
                    for col in date_columns:
                        if col in df.columns:
                            df[col] = df[col].apply(self._format_date)
                            modified = True
                
                # Обработка категорий
                if fill_categories and 'Категория обучения' in df.columns:
                    df['Категория обучения'] = self.default_category
                    modified = True
                
                # Сохраняем файл
                if modified:
                    df.to_excel(target_path, index=False)
                else:
                    shutil.copy2(source_path, target_path)
                
                print(f"✅ {filename}")
                successful += 1
                
            except Exception as e:
                # При ошибке копируем исходный файл
                try:
                    shutil.copy2(source_path, target_path)
                    print(f"⚠️  {filename} (скопирован без изменений)")
                    successful += 1
                except:
                    print(f"❌ {filename} (ошибка: {e})")
        
        print(f"✅ Обработано: {successful}/{len(excel_files)} файлов")
        return successful
    
    def cleanup_temp_files(self, keep_csv=False, keep_excel=False):
        """Очистка временных файлов"""
        cleaned = 0
        
        if not keep_csv and os.path.exists(self.csv_dir):
            try:
                shutil.rmtree(self.csv_dir)
                print(f"🗑️  Удалена папка: {self.csv_dir}")
                cleaned += 1
            except:
                pass
        
        if not keep_excel and os.path.exists(self.excel_dir):
            try:
                shutil.rmtree(self.excel_dir)
                print(f"🗑️  Удалена папка: {self.excel_dir}")
                cleaned += 1
            except:
                pass
        
        return cleaned
    
    def process_complete(self, format_dates=True, fill_categories=True, 
                        cleanup=True, keep_csv=False, keep_excel=False):
        """
        Полная обработка: все этапы от начала до конца
        
        Args:
            format_dates: Форматировать даты
            fill_categories: Заполнять категории обучения
            cleanup: Удалять временные файлы
            keep_csv: Сохранить CSV файлы при очистке
            keep_excel: Сохранить промежуточные Excel файлы при очистке
        
        Returns:
            dict: Статистика обработки
        """
        print(f"🚀 Начало полной обработки файла: {self.input_file}")
        print("=" * 60)
        
        try:
            # Этап 1: Разделение на CSV
            csv_count = self.split_events()
            
            # Этап 2: Конвертация в Excel
            excel_count = self.convert_to_excel()
            
            # Этап 3: Форматирование
            formatted_count = self.format_excel_files(format_dates, fill_categories)
            
            # Очистка временных файлов
            if cleanup:
                print("🔄 Очистка временных файлов...")
                self.cleanup_temp_files(keep_csv, keep_excel)
            
            # Итоговая статистика
            stats = {
                'csv_files': csv_count,
                'excel_files': excel_count,
                'formatted_files': formatted_count,
                'success': True
            }
            
            print("=" * 60)
            print("🎉 ОБРАБОТКА ЗАВЕРШЕНА УСПЕШНО!")
            print(f"📊 Статистика:")
            print(f"   • CSV файлов создано: {csv_count}")
            print(f"   • Excel файлов создано: {excel_count}")
            print(f"   • Файлов отформатировано: {formatted_count}")
            print(f"📁 Результат в папке: {self.final_dir}/")
            
            return stats
            
        except Exception as e:
            print(f"❌ ОШИБКА: {e}")
            return {'success': False, 'error': str(e)}


# Быстрые функции для часто используемых сценариев
def process_events_full(input_file, category="Hard skills", cleanup=True):
    """Полная обработка с настройками по умолчанию"""
    processor = EventProcessorComplete(input_file, default_category=category)
    return processor.process_complete(cleanup=cleanup)

def process_events_keep_temp(input_file, category="Hard skills"):
    """Полная обработка с сохранением всех промежуточных файлов"""
    processor = EventProcessorComplete(input_file, default_category=category)
    return processor.process_complete(cleanup=False)

def process_events_custom(input_file, csv_dir, excel_dir, final_dir, category="Hard skills"):
    """Обработка с пользовательскими папками"""
    processor = EventProcessorComplete(input_file, csv_dir, excel_dir, final_dir, category)
    return processor.process_complete()


def main():
    """Основная функция с примерами использования"""
    # Настройки
    INPUT_FILE = "3.new_code.csv"
    CATEGORY = "Hard skills"
    
    # Вариант 1: Быстрая полная обработка (рекомендуется)
    print("Вариант 1: Полная обработка")
    result = process_events_full(INPUT_FILE, CATEGORY)
    
if __name__ == "__main__":
    main()

Вариант 1: Полная обработка
🚀 Начало полной обработки файла: 3.new_code.csv
🔄 Этап 1: Разделение мероприятий...
✅ Создано 60 CSV файлов
🔄 Этап 2: Конвертация в Excel...


✅ Конвертировано 60 из 60 файлов
🔄 Этап 3: Форматирование Excel файлов...
📁 Найдено файлов: 60
📅 Форматирование дат: включено
🏷️  Заполнение категорий: Hard skills
✅ 1С_Предприятие 8.3_ Администрирование и оптимизация MS SQL Server для поддержки системы 1С_Предприят.xlsx
✅ AL-1707. СУБД PostgreSQL в ОС Astra Linux 1.7_ Установка, администрирование и мониторинг.xlsx
✅ Apache Kafka для разработчиков.xlsx
✅ Data Science. Уровень 1. Инструменты и технологии.xlsx
✅ Data Science. Уровень 2. Применение машинного обучения.xlsx
✅ Data Science. Уровень 3. Масштабируемые решения.xlsx
✅ DBA1 - Администрирование PostgreSQL. Базовый курс.xlsx
✅ DBA2. Администрирование PostgreSQL. Настройка и мониторинг.xlsx
✅ DBA3 - Администрирование PostgreSQL. Резервирование и репликация.xlsx
✅ DEV1. Разработка серверной части приложений PostgreSQL. Базовый курс.xlsx


✅ Figma для UI_UX дизайнера.xlsx
✅ Google Analytics 4 - увеличение эффективности веб-сайтов и рекламы.xlsx
✅ HTML и CSS. Уровень 1. Создание сайтов на HTML и СSS.xlsx
✅ HTML и CSS. Уровень 2. Углубленный CSS и вёрстка макета.xlsx
✅ HTML и CSS. Уровень 3. Продвинутые методологии и инструменты верстки.xlsx
✅ JavaScript. Webpack и сборка JS.xlsx
✅ JavaScript. Уровень 1. Основы JavaScript.xlsx
✅ JavaScript. Уровень 10. HTML5 API.xlsx
✅ JavaScript. Уровень 2. Расширенные возможности.xlsx


✅ JavaScript. Уровень 3. ESNext.xlsx
✅ JavaScript. Уровень 4. Fetch_AJAX API.xlsx
✅ JavaScript. Уровень 5. Серверное программирование на Node.js.xlsx
✅ JavaScript. Уровень 6. Библиотека React.js.xlsx
✅ JavaScript. Уровень 7. Redux и react-router.xlsx
✅ JavaScript. Уровень 8. Vue.js – открытый фреймворк на JavaScript для разработки веб - приложений.xlsx
✅ JavaScript. Уровень 9. Vue.js. Расширенные возможности.xlsx
✅ Linux. Уровень 2. Программирование в Linux на C.xlsx
✅ Microsoft Excel. Уровень 4. Макросы на VBA.xlsx


✅ MySQL 9. Проектирование и создание баз данных.xlsx
✅ Oracle Database_ настройка и оптимизация SQL.xlsx
✅ Oracle Database_ Основы PL_SQL.xlsx
✅ Oracle Database_ Основы SQL.xlsx
✅ PGPRO - Возможности Postgres Pro Enterprise 13.xlsx
✅ PHP8 часть 2. Объектно-ориентированная разработка.xlsx
✅ PHP8 часть 3. Создание сервисов.xlsx
✅ PHP8 часть 4. Профессиональная работа.xlsx
✅ PHP8. Разработка приложений на Laravel.xlsx


✅ PostgreSQL для разработчика. Авторский практикум.xlsx
✅ PostgreSQL_ Уровень 1. Основы SQL.xlsx
✅ PostgreSQL_ Уровень 2. Продвинутые возможности.xlsx
✅ Python для веб-разработки. Flask и проектирование REST API.xlsx
✅ Python для веб-разработки. Flask и разработка веб-приложений.xlsx
✅ Python для машинного обучения.xlsx
✅ QPT - PostgreSQL. Оптимизация запросов.xlsx
✅ Scala 3_ Классы и объектно-ориентированное программирование.xlsx


✅ XML и XSLT. Современные технологии обработки данных для ВЕБ.xlsx
✅ Администрирование веб-сервера Microsoft IIS.xlsx
✅ Веб-сервера Nginx и Apache.xlsx
✅ Защита веб-сайтов от взлома.xlsx
✅ Интернет-маркетинг в поисковых системах_ SEO-оптимизация сайта.xlsx
✅ Интернет-технологии. Базовый курс для маркетологов.xlsx
✅ Клиент-серверная разработка под .Net на языке C#.xlsx
✅ Основы веб-аналитики. Google Analytics 4, Google Tag Manager, Яндекс Метрика.xlsx
✅ Практика создания веб-приложения (фронтенд).xlsx


✅ Практика создания веб-приложения на Laravel (бэкенд).xlsx
✅ Система управления версиями Git.xlsx
✅ Создание лендингов в конструкторе «Тильда».xlsx
✅ Управление проектами внедрения, поддержки, развития интернет-ресурсов и приложений.xlsx
✅ Юзабилити сайтов. Проектирование веб-интерфейсов.xlsx
✅ Яндекс.Метрика. Оценка эффективности сайтов и аналитика рекламы.xlsx
✅ Обработано: 60/60 файлов
🔄 Очистка временных файлов...
🗑️  Удалена папка: split_events
🗑️  Удалена папка: excel_events
🎉 ОБРАБОТКА ЗАВЕРШЕНА УСПЕШНО!
📊 Статистика:
   • CSV файлов создано: 60
   • Excel файлов создано: 60
   • Файлов отформатировано: 60
📁 Результат в папке: 4.excel_final/
